In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

![](https://i.pinimg.com/originals/8e/69/99/8e6999898e0633471f78a67d31778df0.jpg)pinterest.ie

#Fortunately,(for my sake) Kaggle community, very often, choose to help. Otherwise I'd be so screwed.

In [ ]:
#Code By Kohei-mu  https://www.kaggle.com/koheimuramatsu/industrial-machine-anomaly-detection

import numpy as np
import pandas as pd
import holoviews as hv
from holoviews import opts
hv.extension('bokeh')
from matplotlib import pyplot as plt
import seaborn as sns
import os
import scipy.stats as stats
from sklearn.svm import OneClassSVM
from sklearn.ensemble import IsolationForest
from sklearn.neighbors import LocalOutlierFactor
!pip install changefinder
import changefinder
from sklearn.metrics import f1_score
import shap
shap.initjs()
from tabulate import tabulate
from IPython.display import HTML, display

In [ ]:
df = pd.read_csv("/kaggle/input/i-dont-work-here-lady/reddit_i_don_t_work_here_lady.csv")
df.head(2)

In [ ]:
df.isnull().sum()

In [ ]:
df['timestamp'] = pd.to_datetime(df['timestamp'])

#Splitting timestamp columns.

In [ ]:
#Code By Kohei-mu  https://www.kaggle.com/koheimuramatsu/industrial-machine-anomaly-detection

df['year'] = df['timestamp'].apply(lambda x : x.year)
df['month'] = df['timestamp'].apply(lambda x : x.month)
df['day'] = df['timestamp'].apply(lambda x : x.day)
df['hour'] = df['timestamp'].apply(lambda x : x.hour)
df['minute'] = df['timestamp'].apply(lambda x : x.minute)

In [ ]:
df.index = df['timestamp']
df.drop(['timestamp'], axis=1, inplace=True)
df.head(3)

In [ ]:
#Code By Kohei-mu  https://www.kaggle.com/koheimuramatsu/industrial-machine-anomaly-detection

count = hv.Bars(df.groupby(['year','month'])['comms_num'].count()).opts(ylabel="Count", title='Year/Month Count')
mean = hv.Bars(df.groupby(['year','month']).agg({'comms_num': ['mean']})['comms_num']).opts(ylabel="Number of comments", title='Year/Month Mean Comments')
(count + mean).opts(opts.Bars(width=380, height=300,tools=['hover'],show_grid=True, stacked=True, legend_position='bottom'))

In [ ]:
#Code By Kohei-mu  https://www.kaggle.com/koheimuramatsu/industrial-machine-anomaly-detection

year_maxmin = df.groupby(['year','month']).agg({'comms_num': ['min', 'max']})
(hv.Bars(year_maxmin['comms_num']['max']).opts(ylabel="Number of Comments", title='Year/Month Max Comments') \
+ hv.Bars(year_maxmin['comms_num']['min']).opts(ylabel="Number of Comments", title='Year/Month Min Comments'))\
    .opts(opts.Bars(width=380, height=300,tools=['hover'],show_grid=True, stacked=True, legend_position='bottom'))

In [ ]:
#Code By Kohei-mu  https://www.kaggle.com/koheimuramatsu/industrial-machine-anomaly-detection
#Density is the distribution density. Not a feature. Hence, don't change it.

hv.Distribution(df['comms_num']).opts(opts.Distribution(title="Comments Distribution", xlabel="Number of Comments", ylabel="Density", width=700, height=300,tools=['hover'],show_grid=True))

In [ ]:
#Code By Kohei-mu  https://www.kaggle.com/koheimuramatsu/industrial-machine-anomaly-detection

((hv.Distribution(df.loc[df['year']==2021,'comms_num'], label='2021') * hv.Distribution(df.loc[df['year']==2021,'comms_num'], label='2021')).opts(title="Comments by Year Distribution", legend_position='bottom') + \
(hv.Distribution(df.loc[df['month']==12,'comms_num'], label='12') * hv.Distribution(df.loc[df['month']==1,'comms_num'], label='1') \
     * hv.Distribution(df.loc[df['month']==2,'comms_num'], label='2')).opts(title="Comments by Month Distribution", legend_position='bottom')) \
     .opts(opts.Distribution(xlabel="Number of comments", ylabel="Density", width=380, height=300,tools=['hover'],show_grid=True))

In [ ]:
#Code By Kohei-mu  https://www.kaggle.com/koheimuramatsu/industrial-machine-anomaly-detection

hv.Curve(df['comms_num'].resample('D').mean()).opts(opts.Curve(title="Comments Mean by Day", xlabel="Time", ylabel="Number of comments", width=700, height=300,tools=['hover'],show_grid=True))

In [ ]:
#Code By Kohei-Mu  https://www.kaggle.com/koheimuramatsu/women-s-shoes-eda

title_cnt = df['title'].value_counts() > 100
col = hv.Bars(df['title'].value_counts()[title_cnt][::-1]).opts(title="Titles Count over 100", color="purple", xlabel="Titles", ylabel="Count")\
                .opts(opts.Bars(width=400, height=300,tools=['hover'],show_grid=True,invert_axes=True,fontsize={'title':10}))
pr = hv.Distribution(df['score']).opts(title="Score Distribution", color="purple", xlabel="Score", ylabel="Density",xformatter='$ %d')\
                            .opts(opts.Distribution(width=400, height=300,tools=['hover'],show_grid=True))
col + pr

In [ ]:
#Code by Kohei-Mu  https://www.kaggle.com/koheimuramatsu/women-s-shoes-eda

wh_br_cnt = df['title'].value_counts() > 10
br = hv.Bars(df['title'].value_counts()[wh_br_cnt][::-1]).opts(title="Title Count over 10", color="white", xlabel="Titles", ylabel="Count")\
                .opts(opts.Bars(width=400, height=300,tools=['hover'],show_grid=True,invert_axes=True))
pr = hv.Distribution(df['score']).opts(title="Score Distribution", color="white", xlabel="score", ylabel="Density",xformatter='$ %d')\
                            .opts(opts.Distribution(width=400, height=300,tools=['hover'],show_grid=True))
br + pr

#Read  Kohei-Mu https://www.kaggle.com/koheimuramatsu Codes. And vote for his work.

The original code has anomaly. Unfortunately, I couldn't perform Kohei's brilliant code till the end since the only anomaly I have here is myself.

#Kaggle's Meta Meme Mug (MMM)

![](https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcQFtyNUaNDKR-nbu6DF5FmWSJ4SmMugDoIWPg&usqp=CAU)madeindreamland.com